In [1]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import patches
from matplotlib import pyplot as plt
from matplotlib import gridspec as gsp
from matplotlib.widgets import Slider, Button, RadioButtons
from ipywidgets import *

import numpy as np
import csv
import os
import glob
from functools import reduce
import SimpleITK as sitk
from scipy.spatial.distance import cdist

from skimage import measure
from skimage import filters

import time

In [2]:
def get_data(locroot, mtype):
    truthlist = glob.glob(locroot+mtype+'/005/*/predictions/seg-*-tumor.nii')
    truthlist.sort()
    predlist  = glob.glob(locroot+mtype+'/005/*/predictions/pred-*-tumor-masked-int.nii')
    predlist.sort()
    assert len(truthlist) == len(predlist)
    print('Found', len(truthlist), 'segmentations and predictions...')
    return truthlist, predlist


In [14]:
# archs = ['unet', 'resnet', 'densenet']
# pocket = ['', '-pocket']
# c2Dt = ['', '-c2Dt']
archs = ['densenet']
pocket = ['-pocket']
c2Dt = ['']
modeltypes = []
for a in archs:
    for c in c2Dt:
        for p in pocket:
            modeltypes += [a+p+c]

locroot = '/rsrch1/ip/jacctor/smallnet/tumorhcc-ensemble/densenet-ensemble-512/'
# locroot = '/rsrch1/ip/jacctor/smallnet/tumorhcc-ensemble/e2e-512/'

In [4]:
def ShowImage(img):
    data = sitk.GetArrayFromImage(img)
    plt.imshow(data)
    plt.show()

In [5]:
def SliceViewer(data, data2=None):
    img = sitk.GetArrayFromImage(data)
    cmax = np.max(img)
    
    if data2 is not None:
        seg = sitk.GetArrayFromImage(data2)
        cmax = max([cmax, np.max(seg)])

    def f(Slice):
        if data2 is not None:
            fig = plt.figure(figsize=(8,4))
            ax  = fig.add_subplot(121)
        else:
            fig = plt.figure(figsize=(4,4))
            ax  = fig.add_subplot(111)
        img0 = img[...,Slice]
        ax.imshow(img0, cmap="hsv", vmin=0, vmax=cmax)
        if data2 is not None:
            ax = fig.add_subplot(122)
            seg0 = seg[...,Slice]
            ax.imshow(seg0, cmap="hsv", vmin=0, vmax=cmax)
    interact(f, Slice=widgets.IntSlider(min=0, max=img.shape[-1],step=1,value=0))

In [6]:
def closest_node(node, nodes):
    min_idx = cdist([node], nodes).argmin()
    return int(min_idx), nodes[min_idx]

In [7]:
###
### true, pred are SITK::Image
### returns hdist_avg, hdist_max, dice, vol_true
###
def GetSITKStats(true, pred):
    
    # Calculate Hausdorff distances
    try:
        hausdorffFilter = sitk.HausdorffDistanceImageFilter()
#         hausdorffFilter.Execute(true, pred)
#         hdist_avg = hausdorffFilter.GetAverageHausdorffDistance()
#         hdist_max = hausdorffFilter.GetHausdorffDistance()
        hdist_avg = None
        hdist_max = None
    except:
        hdist_avg = None
        hdist_max = None
        
    
    # Calculate Dice scores
    try:
        diceFilter = sitk.LabelOverlapMeasuresImageFilter()
        diceFilter.Execute(true, pred)
        dice = diceFilter.GetDiceCoefficient()
    except:
        dice = None
    
    # Calculate true volume statistics
    try:
        stats = sitk.LabelShapeStatisticsImageFilter()
        stats.Execute(true)
        vol_true = stats.GetNumberOfPixels(1)
    except:
        vol_true = None
    
    results = (hdist_avg, hdist_max, dice, vol_true)
    return results


def GetNpyStats(true, pred):
    if np.sum(pred) == 0:
        dice = 0
    else:
        dice = (2 * np.sum( true * pred ) + 1.0) / ( np.sum(true) + np.sum(pred) + 1.0)
    vol_true = np.sum(true)
    
    results = (None, None, dice, vol_true)
    return results


In [8]:
def connected_components(img):
    labels, n_labels = measure.label(img, return_num=True)
    return labels, n_labels

def get_connected_component_properties(img):
    props = measure.regionprops(img)
    return props

In [44]:
def StatsNPY(truth, pred, iii):
    truth = sitk.ReadImage(truth, sitk.sitkUInt8, imageIO="NiftiImageIO")
    pred  = sitk.ReadImage(pred,  sitk.sitkUInt8, imageIO="NiftiImageIO")
    
    truth = sitk.GetArrayFromImage(truth)
    pred  = sitk.GetArrayFromImage(pred)
    
    global_scores = GetNpyStats(truth, pred)
    individual_scores = {}

    conn_comp_true, num_true_components = connected_components(truth)
    conn_comp_pred, num_pred_components = connected_components(pred)
    
    true_centroids      = np.zeros((num_true_components, 3))
    pred_centroids      = np.zeros((num_pred_components, 3))
    print('\t Found', num_true_components, 'regions in true segmentation.')
    print('\t Found', num_pred_components, 'regions in predicted segmentation.')

    individual_scores['true_vol']     = []
    individual_scores['closest-dice'] = []
#     individual_scores['max-dice']     = []
    individual_scores['hdist_avg']    = []
    individual_scores['hdist_max']    = []

    
    if num_true_components == 0:
        return global_scores, individual_scores, [0.0, 0.0]
    if num_pred_components == 0:
        return global_scores, individual_scores, [0.0, 0.0]
    
        
    conn_comp_true_props = get_connected_component_properties(conn_comp_true)
    conn_comp_pred_props = get_connected_component_properties(conn_comp_pred)
    
    for idx in range(num_true_components):
        true_centroids[idx,:] = np.array( conn_comp_true_props[idx]['centroid'])

    for idx in range(num_pred_components):
        pred_centroids[idx,:] = np.array( conn_comp_pred_props[idx]['centroid'])
        
    
    dsc_matrix  = np.zeros((num_true_components, num_pred_components))
    closest_idx = np.zeros((num_true_components,), dtype=np.int16)
    maxdice_idx = np.zeros((num_true_components,), dtype=np.int16)
    for idx in range(num_true_components):
        this_truth = (conn_comp_true == (idx+1)).astype(np.uint8)
        
        pred_idx, _ = closest_node(true_centroids[idx,:], pred_centroids)
        closest_idx[idx] = pred_idx

        this_pred = (conn_comp_pred == pred_idx).astype(np.uint8)
        this_tumor_stats = GetNpyStats(this_truth, this_pred)

        individual_scores['hdist_avg']    += [this_tumor_stats[0]]
        individual_scores['hdist_max']    += [this_tumor_stats[1]]
        individual_scores['closest-dice'] += [this_tumor_stats[2]]
        individual_scores['true_vol']     += [this_tumor_stats[3]]
        
        for pred_idx in range(num_pred_components):
            this_pred = (conn_comp_pred == (pred_idx+1)).astype(np.uint8)
            this_tumor_stats = GetNpyStats(this_truth, this_pred)
            dsc_matrix[idx, pred_idx] = this_tumor_stats[2]
    
    maxdice_idx = np.argmax(dsc_matrix, axis=-1)
    maxdice     = np.max(dsc_matrix, axis=-1)
    closedice   = np.array( [dsc_matrix[idx, closest_idx[idx]] for idx in range(dsc_matrix.shape[0])])
#     closedice = np.array( individual_scores['closest-dice'])

    print('\t Global dice: {:.4f}'.format(global_scores[2]))
    for idx in range(num_true_components):
        print('\t', idx, '\tDice (closest): {:.4f}'.format(closedice[idx]), '\tDice (max): {:.4f}'.format(maxdice[idx]))
    
#     gs = gsp.GridSpec(1,3, width_ratios=[num_pred_components,1,1])
#     fig = plt.figure()
#     ax1 = plt.subplot(gs[0])
#     ax1.imshow(dsc_matrix, cmap='gray', aspect='equal')
#     ax1.title.set_text('matrix')
#     for idx in range(num_true_components):
#         pred_idx = closest_idx[idx]
#         max_idx  = maxdice_idx[idx]
#         rect = patches.Rectangle((max_idx-0.5,  idx-0.5), 1, 1, edgecolor='g', facecolor="none", linewidth=3)
#         ax1.add_patch(rect)
#         rect = patches.Rectangle((pred_idx-0.5, idx-0.5), 1, 1, edgecolor='r', facecolor="none", linewidth=3)
#         ax1.add_patch(rect)
    
#     ax2 = plt.subplot(gs[1])
#     ax2.imshow(maxdice[:, np.newaxis],   cmap='gray')
#     ax2.title.set_text('max')
#     ax2.set_aspect('equal')
#     ax2.xaxis.set_visible(False)
    
#     ax3 = plt.subplot(gs[2])
#     ax3.imshow(closedice[:, np.newaxis], cmap='gray')
#     ax3.title.set_text('closest')
#     ax3.set_aspect('equal')
#     ax3.xaxis.set_visible(False)
    
#     plt.savefig('dice-matrices-npy/dice-matrix-'+str(iii)+'.png')
#     plt.close()
#     plt.clf()
    
#     for idx in range(num_true_components):
#         individual_scores['max-dice'] += [maxdice[idx]]
    
#     # filter out the false positives, and recompute global scores
#     closest_seg = np.zeros_like(truth, dtype=np.uint8)
#     for idx in range(num_true_components):
#         closest_seg += (conn_comp_pred == (int(closest_idx[idx])+1))
#     closest_seg = (closest_seg > 0).astype(np.uint8)
#     filtered_closest_stats = GetNpyStats(truth, closest_seg)
#     print('\t Filtered dice (closest): {:.4f}'.format(filtered_closest_stats[2]))

        
#     maxreg_seg  = np.zeros_like(truth, dtype=np.uint8)
#     for idx in range(num_true_components):
#         maxreg_seg += (conn_comp_pred == (int(maxdice_idx[idx])+1))
#     maxreg_seg = (maxreg_seg > 0).astype(np.uint8)
#     filtered_maxreg_stats  = GetNpyStats(truth, maxreg_seg)
#     print('\t Filtered dice     (max): {:.4f}'.format(filtered_maxreg_stats[2]))
    
    filtered_scores = [None]*2
#     filtered_scores[0] = filtered_closest_stats[2]
#     filtered_scores[1] = filtered_maxreg_stats[2]
    filtered_scores[0] = closedice
    filtered_scores[1] = maxdice
    
    return global_scores, individual_scores, filtered_scores
#     return global_scores, individual_scores, [None, None]

In [57]:
def StatsNPY2(truth, pred, iii):
    truth = sitk.ReadImage(truth, sitk.sitkUInt8, imageIO="NiftiImageIO")
    pred  = sitk.ReadImage(pred,  sitk.sitkUInt8, imageIO="NiftiImageIO")
    
    truth = sitk.GetArrayFromImage(truth)
    pred  = sitk.GetArrayFromImage(pred)
    
    global_scores = GetNpyStats(truth, pred)
    individual_scores = {}

    conn_comp_true, num_true_components = connected_components(truth)
    conn_comp_pred, num_pred_components = connected_components(pred)
    
    true_centroids      = np.zeros((num_true_components, 3))
    pred_centroids      = np.zeros((num_pred_components, 3))
    print('\t Found', num_true_components, 'regions in true segmentation.')
    print('\t Found', num_pred_components, 'regions in predicted segmentation.')

    individual_scores['true_vol']     = []
    individual_scores['closest-dice'] = []
    individual_scores['hdist_avg']    = []
    individual_scores['hdist_max']    = []

    
    if num_true_components == 0:
        return global_scores, individual_scores, [0.0, 0.0]
    if num_pred_components == 0:
        return global_scores, individual_scores, [0.0, 0.0]
    
        
    conn_comp_true_props = get_connected_component_properties(conn_comp_true)
    conn_comp_pred_props = get_connected_component_properties(conn_comp_pred)
    
    for idx in range(num_true_components):
        true_centroids[idx,:] = np.array( conn_comp_true_props[idx]['centroid'])

    for idx in range(num_pred_components):
        pred_centroids[idx,:] = np.array( conn_comp_pred_props[idx]['centroid'])
        
    
    closest_idx = np.zeros((num_true_components,), dtype=np.int16)
    for idx in range(num_true_components):
        this_truth = (conn_comp_true == (idx+1)).astype(np.uint8)
        
        pred_idx, _ = closest_node(true_centroids[idx,:], pred_centroids)
        closest_idx[idx] = pred_idx

        this_pred = (conn_comp_pred == pred_idx+1).astype(np.uint8)
        this_tumor_stats = GetNpyStats(this_truth, this_pred)

        individual_scores['hdist_avg']    += [this_tumor_stats[0]]
        individual_scores['hdist_max']    += [this_tumor_stats[1]]
        individual_scores['closest-dice'] += [this_tumor_stats[2]]
        individual_scores['true_vol']     += [this_tumor_stats[3]]
        
#         for pred_idx in range(num_pred_components):
#             this_pred = (conn_comp_pred == (pred_idx+1)).astype(np.uint8)
#             this_tumor_stats = GetNpyStats(this_truth, this_pred)
#             dsc_matrix[idx, pred_idx] = this_tumor_stats[2]
    
#     maxdice_idx = np.argmax(dsc_matrix, axis=-1)
#     maxdice     = np.max(dsc_matrix, axis=-1)
#     closedice   = np.array( [dsc_matrix[idx, closest_idx[idx]] for idx in range(dsc_matrix.shape[0])])
    closedice = np.array( individual_scores['closest-dice'])

    print('\t Global dice: {:.4f}'.format(global_scores[2]))
    for idx in range(num_true_components):
        print('\t', idx, '\tDice (closest): {:.4f}'.format(closedice[idx]))
#         print('\t', idx, '\tDice (closest): {:.4f}'.format(closedice[idx]), '\tDice (max): {:.4f}'.format(maxdice[idx]))
    
    
    filtered_scores = [None]*2
    filtered_scores[0] = closedice
    filtered_scores[1] = None
    
    return global_scores, individual_scores, filtered_scores
#     return global_scores, individual_scores, [None, None]

In [58]:
mtype = modeltypes[0]
print('Model type', mtype)
truthlist, predlist = get_data(locroot, mtype)

# truthlist = truthlist[31:33]
# predlist  = predlist[31:33]

Model type densenet-pocket
Found 131 segmentations and predictions...


In [78]:
all_start_time = time.time()
print('Calculating Hausdorff distances...')
hdist_avg = [None]*len(truthlist)
hdist_max = [None]*len(truthlist)
label_sizes = [None]*len(truthlist)
total_sizes = [None]*len(truthlist)
dice_scores = [None]*len(truthlist)
dice_max_scores = [None]*len(truthlist)
dice_closest_scores = [None]*len(truthlist)
individual_stats = [None]*len(truthlist)
for idx, (tr, pr) in enumerate(zip(truthlist, predlist)):
    print('Image', idx)

    start_time = time.time()
    global_scores, individual_scores, filtered_scores = StatsNPY2(tr, pr, idx)
    end_time = time.time()
    elapsed = end_time - start_time

    hdist_avg[idx] = global_scores[0]
    hdist_max[idx] = global_scores[1]
    dice_scores[idx] = global_scores[2]
    total_sizes[idx] = global_scores[3]
    label_sizes[idx] = individual_scores['true_vol']
    individual_stats[idx] = individual_scores
    dice_closest_scores[idx] = filtered_scores[0]
    dice_max_scores[idx] = filtered_scores[1]

    print('\t Time elapsed:\t', elapsed)

all_end_time = time.time()
elapsed = all_end_time - all_start_time
print('\nTotal time elapsed:\t', elapsed)

Calculating Hausdorff distances...
Image 0
	 Found 11 regions in true segmentation.
	 Found 8 regions in predicted segmentation.
	 Global dice: 0.3992
	 0 	Dice (closest): 0.0010
	 1 	Dice (closest): 0.6803
	 2 	Dice (closest): 0.0076
	 3 	Dice (closest): 0.7662
	 4 	Dice (closest): 0.0042
	 5 	Dice (closest): 0.5684
	 6 	Dice (closest): 0.0036
	 7 	Dice (closest): 0.0018
	 8 	Dice (closest): 0.0058
	 9 	Dice (closest): 0.0031
	 10 	Dice (closest): 0.6157
	 Time elapsed:	 5.642416000366211
Image 1
	 Found 12 regions in true segmentation.
	 Found 14 regions in predicted segmentation.
	 Global dice: 0.5824
	 0 	Dice (closest): 0.4099
	 1 	Dice (closest): 0.7818
	 2 	Dice (closest): 0.5376
	 3 	Dice (closest): 0.3579
	 4 	Dice (closest): 0.6370
	 5 	Dice (closest): 0.4314
	 6 	Dice (closest): 0.5175
	 7 	Dice (closest): 0.6541
	 8 	Dice (closest): 0.5356
	 9 	Dice (closest): 0.0023
	 10 	Dice (closest): 0.5348
	 11 	Dice (closest): 0.5698
	 Time elapsed:	 9.874589681625366
Image 2
	 Found

	 Global dice: 0.4426
	 0 	Dice (closest): 0.7963
	 1 	Dice (closest): 0.0001
	 2 	Dice (closest): 0.4876
	 3 	Dice (closest): 0.1504
	 4 	Dice (closest): 0.0001
	 5 	Dice (closest): 0.6950
	 6 	Dice (closest): 0.6775
	 Time elapsed:	 8.93180251121521
Image 29
	 Found 1 regions in true segmentation.
	 Found 1 regions in predicted segmentation.
	 Global dice: 0.0001
	 0 	Dice (closest): 0.0001
	 Time elapsed:	 6.849960088729858
Image 30
	 Found 1 regions in true segmentation.
	 Found 12 regions in predicted segmentation.
	 Global dice: 0.0256
	 0 	Dice (closest): 0.0206
	 Time elapsed:	 7.925718545913696
Image 31
	 Found 8 regions in true segmentation.
	 Found 7 regions in predicted segmentation.
	 Global dice: 0.5943
	 0 	Dice (closest): 0.0013
	 1 	Dice (closest): 0.3308
	 2 	Dice (closest): 0.7939
	 3 	Dice (closest): 0.7935
	 4 	Dice (closest): 0.0010
	 5 	Dice (closest): 0.1706
	 6 	Dice (closest): 0.5429
	 7 	Dice (closest): 0.0057
	 Time elapsed:	 6.665682315826416
Image 32
	 Fou

	 Found 1 regions in true segmentation.
	 Found 4 regions in predicted segmentation.
	 Global dice: 0.0003
	 0 	Dice (closest): 0.0004
	 Time elapsed:	 3.813220977783203
Image 55
	 Found 1 regions in true segmentation.
	 Found 34 regions in predicted segmentation.
	 Global dice: 0.0306
	 0 	Dice (closest): 0.0533
	 Time elapsed:	 8.205090999603271
Image 56
	 Found 2 regions in true segmentation.
	 Found 15 regions in predicted segmentation.
	 Global dice: 0.8234
	 0 	Dice (closest): 0.7603
	 1 	Dice (closest): 0.8308
	 Time elapsed:	 11.34029245376587
Image 57
	 Found 2 regions in true segmentation.
	 Found 5 regions in predicted segmentation.
	 Global dice: 0.7780
	 0 	Dice (closest): 0.8348
	 1 	Dice (closest): 0.7183
	 Time elapsed:	 18.708940267562866
Image 58
	 Found 5 regions in true segmentation.
	 Found 2 regions in predicted segmentation.
	 Global dice: 0.0018
	 0 	Dice (closest): 0.0144
	 1 	Dice (closest): 0.0081
	 2 	Dice (closest): 0.0052
	 3 	Dice (closest): 0.0017
	 4 	D

	 Found 1 regions in true segmentation.
	 Found 7 regions in predicted segmentation.
	 Global dice: 0.7365
	 0 	Dice (closest): 0.8188
	 Time elapsed:	 13.360686302185059
Image 87
	 Found 1 regions in true segmentation.
	 Found 33 regions in predicted segmentation.
	 Global dice: 0.7136
	 0 	Dice (closest): 0.3653
	 Time elapsed:	 20.083772897720337
Image 88
	 Found 1 regions in true segmentation.
	 Found 0 regions in predicted segmentation.
	 Time elapsed:	 31.294018983840942
Image 89
	 Found 11 regions in true segmentation.
	 Found 110 regions in predicted segmentation.
	 Global dice: 0.1886
	 0 	Dice (closest): 0.0139
	 1 	Dice (closest): 0.0001
	 2 	Dice (closest): 0.0001
	 3 	Dice (closest): 0.1169
	 4 	Dice (closest): 0.0639
	 5 	Dice (closest): 0.0005
	 6 	Dice (closest): 0.0017
	 7 	Dice (closest): 0.0940
	 8 	Dice (closest): 0.0605
	 9 	Dice (closest): 0.3866
	 10 	Dice (closest): 0.0002
	 Time elapsed:	 63.81490182876587
Image 90
	 Found 2 regions in true segmentation.
	 Foun

	 Found 28 regions in true segmentation.
	 Found 18 regions in predicted segmentation.
	 Global dice: 0.0548
	 0 	Dice (closest): 0.0006
	 1 	Dice (closest): 0.1903
	 2 	Dice (closest): 0.4634
	 3 	Dice (closest): 0.3788
	 4 	Dice (closest): 0.0013
	 5 	Dice (closest): 0.0039
	 6 	Dice (closest): 0.0016
	 7 	Dice (closest): 0.0007
	 8 	Dice (closest): 0.3393
	 9 	Dice (closest): 0.3870
	 10 	Dice (closest): 0.0086
	 11 	Dice (closest): 0.0001
	 12 	Dice (closest): 0.0054
	 13 	Dice (closest): 0.1653
	 14 	Dice (closest): 0.2749
	 15 	Dice (closest): 0.0026
	 16 	Dice (closest): 0.0002
	 17 	Dice (closest): 0.0191
	 18 	Dice (closest): 0.0003
	 19 	Dice (closest): 0.0547
	 20 	Dice (closest): 0.0250
	 21 	Dice (closest): 0.0007
	 22 	Dice (closest): 0.0021
	 23 	Dice (closest): 0.0023
	 24 	Dice (closest): 0.0309
	 25 	Dice (closest): 0.0010
	 26 	Dice (closest): 0.0012
	 27 	Dice (closest): 0.0018
	 Time elapsed:	 128.46685600280762
Image 110
	 Found 17 regions in true segmentation.
	 

	 Found 1 regions in true segmentation.
	 Found 3 regions in predicted segmentation.
	 Global dice: 0.6810
	 0 	Dice (closest): 0.6433
	 Time elapsed:	 16.852285623550415
Image 126
	 Found 1 regions in true segmentation.
	 Found 2 regions in predicted segmentation.
	 Global dice: 0.1332
	 0 	Dice (closest): 0.1349
	 Time elapsed:	 16.31138563156128
Image 127
	 Found 1 regions in true segmentation.
	 Found 2 regions in predicted segmentation.
	 Global dice: 0.0010
	 0 	Dice (closest): 0.0011
	 Time elapsed:	 49.203855752944946
Image 128
	 Found 11 regions in true segmentation.
	 Found 43 regions in predicted segmentation.
	 Global dice: 0.3884
	 0 	Dice (closest): 0.1356
	 1 	Dice (closest): 0.0151
	 2 	Dice (closest): 0.0817
	 3 	Dice (closest): 0.0316
	 4 	Dice (closest): 0.0003
	 5 	Dice (closest): 0.5783
	 6 	Dice (closest): 0.0007
	 7 	Dice (closest): 0.3487
	 8 	Dice (closest): 0.0002
	 9 	Dice (closest): 0.4581
	 10 	Dice (closest): 0.0490
	 Time elapsed:	 57.05408310890198
Image

In [60]:
print(dice_closest_scores[0])

[0.0010142  0.68025258 0.00757576 0.76623377 0.0041841  0.56838906
 0.00357143 0.00184162 0.00578035 0.0031348  0.61568627]


In [ ]:
plt.figure(figsize=(12,4))
plt.scatter(list(range(len(dice_scores))), dice_scores, c='black')
plt.scatter(list(range(len(dice_closest_scores))), dice_closest_scores, c='r')
plt.scatter(list(range(len(dice_max_scores))), dice_max_scores, c='g')
plt.savefig('global-tumor-dice-results-npy.png')
plt.close()
plt.clf()

plt.figure(figsize=(4,4))
plt.hist(each_tumor_volume, bins=100, range=[0,20000])
plt.savefig('tumor-volume-hist-npy.png')
plt.close()
plt.clf()

In [65]:
dicediff_closest = np.array(dice_closest_scores) - np.array(dice_scores)
dicediff_max     = np.array(dice_max_scores)     - np.array(dice_scores)

dicediff_closest = dicediff_closest[np.isfinite(dicediff_closest)]
dicediff_max     = dicediff_max[np.isfinite(dicediff_max)]

plt.figure(figsize=(8,4))
ax = plt.subplot(121)
plt.hist(dicediff_closest, bins=50)
ax.set_title('Closest Centroid')
ax = plt.subplot(122)
plt.hist(dicediff_max, bins=50)
ax.set_title('Max Dice')
plt.savefig('global-tumor-dice-difference-filtering-npy.png')
plt.close()
plt.clf()

/opt/apps/miniconda/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

In [85]:
    
# Individual Dice Statistics
each_tumor_dice      = []
each_tumor_dice_max  = []
each_tumor_volume    = []
for idx in range(len(truthlist)):
    if individual_stats[idx] is not None:
        each_tumor_dice      += individual_stats[idx]['closest-dice']
#         each_tumor_dice_max  += individual_stats[idx]['max-dice']
        each_tumor_volume    += individual_stats[idx]['true_vol']
print(len(each_tumor_dice))
print(len(each_tumor_volume))
print('Number of zero dice scores:', len([v for v in each_tumor_dice     if v <= 0.01]))
# print('Number of zero dice scores:', len([v for v in each_tumor_dice_max if v <= 0.01]))


plt.figure(figsize=(8,4))
# ax = plt.subplot(121)
ax = plt.subplot(111)
plt.scatter( each_tumor_volume, each_tumor_dice)
ax.set_title('Volume vs. Dice of Closest Centroid')
plt.xlim([0,7.e4])
# ax = plt.subplot(122)
# plt.scatter( each_tumor_volume, each_tumor_dice_max)
# ax.set_title('Volume vs. Dice of Max Predicted Region')
# plt.xlim(0,7.e4)
plt.savefig('individual-tumor-dice-results-npy.png')
plt.close()
plt.clf()
# plt.show()

879
879
Number of zero dice scores: 449


In [88]:
plt.figure(figsize=(2,4))
plt.boxplot(each_tumor_dice)
plt.title('Dice per Tumor')
plt.savefig('boxplot-individual-tumor-dice-results-npy.png')
plt.close()
plt.clf()


In [93]:
fig, ax = plt.subplots()
ax.violinplot(each_tumor_dice)
ax.set_title('Dice per Tumor')
plt.savefig('violin-individual-tumor-dice-results-npy.png')
plt.close()
plt.clf()


In [82]:
plt.figure(figsize=(4,4))
plt.scatter( each_tumor_volume, each_tumor_dice)
plt.title('Volume vs. Dice of Closest Centroid')
plt.xlim([0,7.e4])
plt.savefig('individual-tumor-dice-results-npy.png')
plt.close()
plt.clf()


In [83]:
plt.figure(figsize=(4,4))
plt.boxplot([d for d in dice_scores if d is not None])
plt.title('Global Tumor DSC')
plt.savefig('boxplot-global-tumor-dice-results-npy.png')
plt.close()
plt.clf()


In [86]:
print(dice_scores)

[0.3992443324937028, 0.582386187193342, 0.4453857705494007, 0.38544902169723627, 0.02736669015738783, 0.020396979663145347, 0.2622556624263109, 0.004483362088690178, 0.7970489138997068, 0.6684473428611073, 0.5970313647330326, 0.056139784706920985, 0.7145187058980162, 0.02850610041793954, 0.6028131384567028, 0.543731089258699, 0.6252806444463717, 0.2081864473168821, 0.30227518959913324, 0.583809908273341, 0.7499089253187614, 0.7873708959035716, 0.000363901018922853, 0.6794916707940227, 0.7204240082249087, 0.5925786363095474, 0.4513785912882298, 0.6781255211837846, 0.4425679692800537, 6.559958016268695e-05, 0.02558550596553248, 0.5943145084616848, 0, 0.7111355696707966, 2.296169071521074e-06, 1.9658724541951718e-05, 0.5134853804803778, 0.7784403830638089, 0.0008143322475570033, 0.0014124695196466837, 0.6249330286364083, 0.0010235414534288639, 0.0004042037186742118, 0.0003824941769197642, 0.5356380341997103, 1.1875920383829747e-05, 0.5695476492141218, 0.002028397565922921, 0.8027015127213

In [87]:
print(len([d for d in dice_scores if d > 0.05]))

84


In [96]:
large_vols = np.array(each_tumor_volume) > 2000

fig, ax = plt.subplots()
ax.violinplot(np.array(each_tumor_dice)[large_vols])
ax.set_title('Dice per Tumor')
plt.savefig('violin-individual-tumor-dice-results-npy-largervol.png')
plt.close()
plt.clf()

plt.figure()
plt.boxplot(np.array(each_tumor_dice)[large_vols])
plt.title('Dice of Largest Tumors')
plt.savefig('boxplot-individual-tumor-dice-results-npy-largest.png')
plt.close()
plt.clf()


In [97]:
np.average(dice_scores)

0.3128009050962935

In [99]:
plt.figure()
plt.hist(dice_scores, bins=20, range=[0,1])
plt.title('Global Tumor Dice Scores')
plt.savefig('hist-global-tumor-dice.png')
plt.close()
plt.clf()

In [100]:
plt.figure()
plt.hist(each_tumor_dice, bins=20, range=[0,1])
plt.title('Individual Tumor Dice Scores')
plt.savefig('hist-indiv-tumor-dice.png')
plt.close()
plt.clf()

In [101]:
plt.figure()
plt.hist(np.array(each_tumor_dice)[large_vols], bins=20, range=[0,1])
plt.title('Individual Tumor Dice Scores - Largest Tumors')
plt.savefig('hist-indiv-tumor-dice-largest.png')
plt.close()
plt.clf()

# Liver results

In [111]:
def get_data(locroot, mtype):
    truthlist = glob.glob(locroot+mtype+'/005/*/predictions/seg-*-liver.nii')
    truthlist.sort()
    predlist  = glob.glob(locroot+mtype+'/005/*/predictions/pred-*liver-lcc.nii')
    predlist.sort()
    assert len(truthlist) == len(predlist)
    print('Found', len(truthlist), 'segmentations and predictions...')
    return truthlist, predlist

def initialize_dict(textfile, n_expected=7):
    with open(textfile, 'r') as f:
        linelist = f.readlines()
        testscores = [line for line in linelist if line[0]=='+']
        individual_scores = dict()
        try:
            lines = testscores[:n_expected]
            for l in lines:
                wordlist = l.split()
                name = reduce(lambda x, y: x+' '+y, wordlist[1:-2])
                individual_scores[name] = []
        except:
            print('cannot process file to initialize dict')
        return individual_scores

def process_textfile(textfilelist, score_dict):
    for textfile in textfilelist:
        with open(textfile, 'r') as f:
            linelist = f.readlines()
            testscores = [line for line in linelist if line[0]=='+']
#             try:
            for line in testscores:
                    splitline = line.split()
                    name = reduce(lambda x, y: x+' '+y, splitline[1:-2])
                    score_dict[name] += [float(splitline[-1])]
#                 metrics = [m[1:-2] for m in testscores[0][1:].split()]
#                 assert list(score_dict.keys()) == metrics
#                 for test_img in testscores[2:]:
#                     scores = [float(s[:-1]) for s in test_img[1:].split()]
#                     for i in range(len(metrics)):
#                         score_dict[metrics[i]] += [scores[i]] 
#             except:
#                 print('cannot process file:', textfile)
    return score_dict

def gather_test_scores(textfilelist):
    score_dict = initialize_dict(textfilelist[0])
    return process_textfile(textfilelist, score_dict)

def collate_model_comparisons(mtype, locroot='', separate_folds=False):
    print('\ncollating stats for base model', mtype)
    
    textfilelist = glob.glob(locroot+mtype+'/output-005-*.txt')
    print('number of files found:\t', len(textfilelist), '\n')
    
    if separate_folds:
        textfilelist_brokenstrings = [f.split('\\') for f in textfilelist]
    else:
        textfilelist_brokenstrings = [f.split('/') for f in textfilelist]
    textfilelist_dict = {}
    for idx, t in enumerate(textfilelist_brokenstrings):
        try:
            textfilelist_dict[t[0]] += [textfilelist[idx]]
        except:
            textfilelist_dict[t[0]] = [textfilelist[idx]]

    for model in textfilelist_dict:
        print(model)
        scores_dict = gather_test_scores(textfilelist_dict[model])
        plt.figure(figsize=(12,2))
        for idx, metric in enumerate(scores_dict):
            print(metric, np.mean(np.array(scores_dict[metric])), np.std(np.array(scores_dict[metric])))
            ax = plt.subplot(1,7,idx+1)
            plt.hist(scores_dict[metric], range=[0,1], bins=25)
            ax.set_title(metric)
#         plt.show()
        plt.savefig('hist-dsc-liver.png')
        plt.close()
        plt.clf()
        print()
    return scores_dict

In [112]:
# modeltypes = ['unet', 'resnet', 'densenet']
results_dict = {}
for mtype in modeltypes:
    results_dict[mtype] = collate_model_comparisons(mtype, separate_folds=False, locroot=locroot)


collating stats for base model densenet-pocket
number of files found:	 5 


DSC-L2 3D LIVER LCC (int) 0.11618590136829153 0.10887101714475207
DSC-L2 3D TUMOR MASKED (float) 0.6663702872625922 0.3104363167587569
DSC-L2 3D TUMOR MASKED (int) 0.6721255835111879 0.30524834272059226
DSC-L2 3D LIVER (float) 0.1587420640662336 0.10222564522148304
DSC-L2 3D LIVER (int) 0.18335813677325144 0.10691123736684387
DSC-L2 3D TUMOR (float) 0.7318251174624842 0.28201252654602327
DSC-L2 3D TUMOR (int) 0.7351669194103679 0.2802235033888865



In [116]:
plt.figure()
plt.boxplot( 1 - np.array(results_dict['densenet-pocket']['DSC-L2 3D LIVER LCC (int)']))
plt.title('Liver DSC')
plt.savefig('boxplot-liver-dsc-npy.png')
plt.close()
plt.clf()

In [119]:
np.average(  [i for i in results_dict['densenet-pocket']['DSC-L2 3D TUMOR MASKED (int)'] if i < 0.95]) 

0.5032702008277529